<a href="https://colab.research.google.com/github/rishabhjohri/SarcasmDetection_renforcementlearning/blob/main/DeepLearningReview2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download rmisra/news-headlines-dataset-for-sarcasm-detection
! unzip news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 200MB/s]
Archive:  news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
%matplotlib inline
import os
import re
import tensorflow as tf


In [4]:
import json
def parse_data(file):
    for l in open(file,'r'):
        yield json.loads(l)

data = list(parse_data('/content/Sarcasm_Headlines_Dataset.json'))
data2 = list(parse_data('/content/Sarcasm_Headlines_Dataset_v2.json'))

In [5]:
df=pd.DataFrame(data)
df2=pd.DataFrame(data2)

In [6]:
df = df[['headline','is_sarcastic']]
df.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [7]:
df2 = df2[['headline','is_sarcastic']]
df2.head()

,headline,is_sarcastic
0,thirtysomething scientists unveil doomsday clo...,1
1,dem rep. totally nails why congress is falling...,0
2,eat your veggies: 9 deliciously different recipes,0
3,inclement weather prevents liar from getting t...,1
4,mother comes pretty close to using word 'strea...,1


In [8]:
df['headline'] = df['headline'].apply(lambda x: x.lower())
df['headline'] = df['headline'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [9]:
df['headline']

0        former versace store clerk sues over secret bl...
1        the roseanne revival catches up to our thorny ...
2        mom starting to fear sons web series closest t...
3        boehner just wants wife to listen not come up ...
4        jk rowling wishes snape happy birthday in the ...
                               ...                        
26704                  american politics in moral freefall
26705                               americas best 20 hikes
26706                                reparations and obama
26707    israeli ban targeting boycott supporters raise...
26708                    gourmet gifts for the foodie 2014
Name: headline, Length: 26709, dtype: object

In [10]:
df2['headline'] = df2['headline'].apply(lambda x: x.lower())
df2['headline'] = df2['headline'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [11]:
vocab_size = 10000
embedding_dim = 16
max_length = 100

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(df2['headline'])
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(df2['headline'])
training_padded = pad_sequences(training_sequences,maxlen=max_length, padding='post', truncating='post')

testing_sequences = tokenizer.texts_to_sequences(df['headline'])
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding='post', truncating='post')

In [12]:
training_padded

array([[   1,  337, 3106, ...,    0,    0,    0],
       [7348, 1732,  733, ...,    0,    0,    0],
       [ 893,   35,    1, ...,    0,    0,    0],
       ...,
       [   4,   97,  590, ...,    0,    0,    0],
       [1830, 1284, 3255, ...,    0,    0,    0],
       [ 210, 3224,   21, ...,    0,    0,    0]], dtype=int32)

In [13]:
training_padded = np.array(training_padded)
training_labels = np.array(df2['is_sarcastic'])
testing_padded = np.array(testing_padded)
testing_labels = np.array(df['is_sarcastic'])

In [14]:
#LSTM MODEL
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(training_padded, training_labels, epochs=30, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/30
895/895 - 73s - loss: 0.3895 - accuracy: 0.8163 - val_loss: 0.2165 - val_accuracy: 0.9171 - 73s/epoch - 82ms/step
Epoch 2/30
895/895 - 70s - loss: 0.2262 - accuracy: 0.9094 - val_loss: 0.1466 - val_accuracy: 0.9531 - 70s/epoch - 78ms/step
Epoch 3/30
895/895 - 68s - loss: 0.1568 - accuracy: 0.9407 - val_loss: 0.0878 - val_accuracy: 0.9735 - 68s/epoch - 76ms/step
Epoch 4/30
895/895 - 63s - loss: 0.1050 - accuracy: 0.9621 - val_loss: 0.0606 - val_accuracy: 0.9838 - 63s/epoch - 70ms/step
Epoch 5/30
895/895 - 66s - loss: 0.0673 - accuracy: 0.9775 - val_loss: 0.0395 - val_accuracy: 0.9882 - 66s/epoch - 74ms/step
Epoch 6/30
895/895 - 71s - loss: 0.0474 - accuracy: 0.9841 - val_loss: 0.0257 - val_accuracy: 0.9929 - 71s/epoch - 80ms/step
Epoch 7/30
895/895 - 64s - loss: 0.0321 - accuracy: 0.9900 - val_loss: 0.0201 - val_accuracy: 0.9936 - 64s/epoch - 71ms/step
Epoch 8/30
895/895 - 64s - loss: 0.0258 - accuracy: 0.9912 - val_loss: 0.0151 - val_accuracy: 0.9959 - 64s/epoch - 71ms/step


In [15]:
score,acc=model.evaluate(testing_padded,testing_labels)

835/835 [==============================] - 15s 18ms/step - loss: 5.1904e-04 - accuracy: 0.9999


In [16]:
acc,score

(0.9998877048492432, 0.0005190441734157503)

In [18]:
import pickle

# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))


INFO:tensorflow:Assets written to: ram://eb94d97c-c403-4561-8ee7-206fd1440d5e/assets


INFO:tensorflow:Assets written to: ram://eb94d97c-c403-4561-8ee7-206fd1440d5e/assets


In [19]:
loaded_model